In [1]:
from suffix_tree import Tree
#lexicographic order
def leqPair(a1, a2, b1, b2): 
   return a1 < b1 or a1 == b1 and a2 <= b2
def leqTriple(a1, a2, a3, b1, b2, b3 ):
   return a1 < b1 or a1 == b1 and leqPair(a2,a3,b2,b3)

In [2]:
def radixSort(a, b, r, offset, length, alphabetSize):
    count = [0 for _ in range(alphabetSize + 1)] # counter array
    for i in range(0, length):
        count[r[a[i]+offset]] += 1 # count occurrences
    i = 0
    sum = 0
    while i <= alphabetSize: # exclusive prefix sums
        t = count[i]
        count[i] = sum
        sum += t
        i += 1
    for i in range(0, length): #sort
        b[count[r[a[i]+offset]]] = a[i]
        count[r[a[i]+offset]] += 1


In [3]:
class suffix_array:
    def naiveConstruct(self,string):
        string+='$'
        self.array = []
        for i in range(len(string)):
            self.array.append((string[i:],i))
        self.array=sorted(self.array)
        self.array = [x[1] for x in self.array]
        self.array.pop(0)
        self.size = len(self.array)
        self.string=string[:-1]
    
    def cyclicConstruct(self,string):
        string+='$'
        self.size = len(string)
        self.string = string
        alphabet = 256
        p = [0]*self.size
        c = [0]*self.size
        cnt = [0]*max(alphabet,self.size)

        for i in range(0,self.size):
            cnt[ord(string[i])]+=1
        for i in range(1,alphabet):
            cnt[i]+=cnt[i-1]
        for i in range(0,self.size):
            cnt[ord(string[i])]-=1
            p[cnt[ord(string[i])]] = i

        c[p[0]]=0
        classes = 1
        for i in range(1,self.size):
            if string[p[i]]!=string[p[i-1]]:
                classes+=1
            c[p[i]] = classes-1


        pn = [0]*self.size
        cn = [0]*self.size

        h=0
        while (1 << h) < self.size:
            for i in range(self.size):
                pn[i] = p[i] - (1<<h)
                if pn[i] < 0:
                    pn[i]+=self.size
            #fill
            cnt[:classes] = [0]*classes
            for i in range(0,self.size):
                cnt[c[pn[i]]]+=1
            for i in range(1,classes):
                cnt[i]+=cnt[i-1]
            for i in range(self.size-1,-1,-1):
                cnt[c[pn[i]]]-=1
                p[cnt[c[pn[i]]]] = pn[i]
            
            c[p[0]] = 0
            classes = 1
            for i in range(1,self.size):
                cur = (c[p[i]], c[(p[i] + (1 << h)) % self.size])
                prev = (c[p[i-1]], c[(p[i-1] + (1 << h)) % self.size])
                if cur!=prev:
                    classes+=1
                cn[p[i]] = classes -1
            c,cn=cn,c
            h+=1
        self.classes = c
        self.array=p
        self.array.pop(0)
        self.size = len(self.array)
    
    def skewConstruct(self,s):
        self.string = s
        string = [ord(c) for c in s]
        sentinelCharacter = -1
        #string.append(sentinelCharacter)
        sentinelCharacter-=1
        numStrings = 1
        
        self.array=[0 for _ in range(len(string))]

        #s[n]=s[n+1]=s[n+2]=0, n>=2
        string.append(0)
        string.append(0)
        string.append(0)
        self.skewSuffixArray(string, self.array, len(string)-3, 256)
        self.size = len(self.array)
        #self.array.pop(0)


    #find the suffix array SA of s[0..n-1] in {1..K}ˆn
    #require s[n]=s[n+1]=s[n+2]=0, n>=2
    def skewSuffixArray(self,s, SA, length, alphabetSize):
        n0 = (length + 2) // 3
        n1 = (length + 1) // 3
        n2 = length // 3
        n02 = n0 + n2
        s12 = [0 for _ in range(n02 + 3)]
        s12[n02] = s12[n02 + 1] = s12[n02 + 2] = 0
        SA12 = [0 for _ in range(n02 + 3)]
        SA12[n02] = SA12[n02 + 1] = SA12[n02 + 2] = 0
        s0 = [0 for _ in range(n0)]
        SA0 = [0 for _ in range(n0)]


        i = 0
        j = 0
        #generate positions of mod 1 and mod 2 suffixes
        #the "+(n0-n1)" adds a dummy mod 1 suffix if n%3 == 1
        while i < length + (n0 - n1):
            if (i%3) != 0:
                s12[j] = i
                j += 1
            i += 1

        radixSort(s12, SA12, s, 2, n02, alphabetSize)
        radixSort(SA12, s12, s, 1, n02, alphabetSize)
        radixSort(s12, SA12, s, 0, n02, alphabetSize)

        name = 0
        c0 = -1
        c1 = -1
        c2 = -1
        #find lexicographic names of triples
        for i in range(0, n02):
            if s[SA12[i]] != c0 or s[SA12[i] + 1] != c1 or s[SA12[i] + 2] != c2:
                name += 1
                c0 = s[SA12[i]]
                c1 = s[SA12[i] + 1]
                c2 = s[SA12[i] + 2]
            if (SA12[i]% 3) == 1:
                s12[SA12[i] // 3] = name
            else:
                s12[(SA12[i] // 3) + n0] = name
        #recurse if names are not yet unique
        if name < n02:
            self.skewSuffixArray(s12, SA12, n02, name)
            #store unique names in s12 using the suffix array
            for i in range(0, n02):
                s12[SA12[i]] = i + 1
        else: #generate the suffix array of s12 directly
            for i in range(0, n02):
                SA12[s12[i] - 1] = i

        i = 0
        j = 0
        #stably sort the mod 0 suffixes from SA12 by their first character
        while i < n02:
            if SA12[i] < n0:

                s0[j] = 3 * SA12[i]
                j += 1
            i += 1
        radixSort(s0, SA0, s, 0, n0, alphabetSize)

        p = 0
        t = n0 - n1
        k = 0
        #merge sorted SA0 suffixes and sorted SA12 suffixes
        while k < length:
            i = SA12[t] * 3 + 1 if SA12[t] < n0 else (SA12[t] - n0) * 3 + 2 #offset of 12 suffix
            j = SA0[p] #offset of 0 suffix
            if leqPair(s[i],s12[SA12[t]+n0],s[j],s12[(j // 3)]) if SA12[t] < n0 else leqTriple(s[i],s[i+1],s12[SA12[t]-n0+1], s[j],s[j+1],s12[(j // 3)+n0]):
                SA[k] = i
                t += 1
                if t == n02:#done --- only SA0 suffixes left
                    k = k + 1
                    while p < n0:
                        SA[k] = SA0[p]
                        p += 1
                        k += 1
            else:# suffix from SA0 is smaller
                SA[k] = j
                p += 1
                if p == n0:# done --- only SA12 suffixes left
                    k = k + 1
                    while t < n02:
                        SA[k] = SA12[t] * 3 + 1 if SA12[t] < n0 else (SA12[t] - n0) * 3 + 2
                        t += 1
                        k += 1
            k += 1

    def suffixAt(self,position):
        return self.string[position:]
    
    def translate(self,interval):
        positions = self.array[interval[0]:interval[1]]
        return [self.string[i:] for i in positions]

    def search(self,substring):
        #starting position of interval
        l=0
        r=self.size
        while l<r:
            mid = (l + r) // 2 
            if substring > self.suffixAt(self.array[mid]):
                l = mid+1
            else:
                r = mid
        return l
        
    def searchAll(self,substring):
        #starting position of interval
        l=0
        r=self.size
        while l<r:
            mid = (l + r) // 2 
            if substring > self.suffixAt(self.array[mid]):
                l = mid+1
            else:
                r = mid
        s=l

        #ending position of interval
        r = self.size
        while l < r:
            mid = (l + r) // 2
            if self.suffixAt(self.array[mid]).startswith(substring):
                l = mid + 1
            else:
                r = mid
        e=r
        return (s, e)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import time
from suffix_tree import naive
from suffix_tree import ukkonen

def testRandomString(size):
    constructionTimes = {}
    np.random.seed(0)
    #create random string
    #start after $(ascii code 36)
    string = ''.join([chr(x) for  x in np.random.randint(37,256,(size))])
    #time each creation, starting from naive
    creationStart = time.time()
    s = suffix_array()
    s.naiveConstruct(string)
    constructionTimes['SA-Naive'] = time.time()-creationStart

    creationStart = time.time()
    s = suffix_array()
    s.cyclicConstruct(string)
    constructionTimes['SA-Cyclic'] = time.time()-creationStart

    creationStart = time.time()
    s = suffix_array()
    s.skewConstruct(string)
    constructionTimes['SA-Skew'] = time.time()-creationStart

    creationStart = time.time()
    s = Tree({"A": string},builder=ukkonen.Builder)
    constructionTimes['ST-Ukkonen'] = time.time()-creationStart

    creationStart = time.time()
    s = Tree({"A": string},builder=naive.Builder)
    constructionTimes['ST-Naive'] = time.time()-creationStart


    return constructionTimes

In [7]:
def queryOne(queries=10000,size=10000):
    np.random.seed(0)
    string = ''.join([chr(x) for  x in np.random.randint(37,256,(size))])
    s = suffix_array()
    s.skewConstruct(string)
    np.random.seed(0)
    startTime = time.time()
    for _ in range(queries):
        startIndex = np.random.randint(size)
        endIndex = np.random.randint(startIndex,size)
        substring = string[startIndex:endIndex+1]
        s.search(substring)
    saTime = time.time()-startTime

    s=Tree({'A':string})
    np.random.seed(0)
    startTime = time.time()
    for _ in range(queries):
        startIndex = np.random.randint(size)
        endIndex = np.random.randint(startIndex,size)
        substring = string[startIndex:endIndex+1]
        s.find(substring)
    stTime = time.time()-startTime

    return saTime,stTime

def queryAll(queries=10000,size=10000):
    np.random.seed(0)
    string = ''.join([chr(x) for  x in np.random.randint(37,256,(size))])
    s = suffix_array()
    s.skewConstruct(string)
    np.random.seed(0)
    startTime = time.time()
    for _ in range(queries):
        startIndex = np.random.randint(size)
        endIndex = np.random.randint(startIndex,size)
        substring = string[startIndex:endIndex+1]
        s.searchAll(substring)
    saTime = time.time()-startTime

    s=Tree({'A':string})
    np.random.seed(0)
    startTime = time.time()
    for _ in range(queries):
        startIndex = np.random.randint(size)
        endIndex = np.random.randint(startIndex,size)
        substring = string[startIndex:endIndex+1]
        s.find_all(substring)
    stTime = time.time()-startTime

    return saTime,stTime

def queryFixed(queries=10000,size=10000):
    np.random.seed(0)
    string = ''.join([chr(x) for  x in np.random.randint(37,256,(size))])
    s = suffix_array()
    s.skewConstruct(string)
    np.random.seed(0)
    startTime = time.time()
    for _ in range(queries):
        startIndex = np.random.randint(size)
        endIndex = startIndex+5
        substring = string[startIndex:endIndex+1]
        s.searchAll(substring)
    saTime = time.time()-startTime

    s=Tree({'A':string})
    np.random.seed(0)
    startTime = time.time()
    for _ in range(queries):
        startIndex = np.random.randint(size)
        endIndex = startIndex+5
        substring = string[startIndex:endIndex+1]
        s.find_all(substring)
    stTime = time.time()-startTime

    return saTime,stTime

In [10]:
d={}
for size in [100,1000,10000,100000]:
    d[size] = testRandomString(size)

In [15]:
import pandas as pd
pd.DataFrame(d).transpose()

,SA-Naive,SA-Cyclic,SA-Skew,ST-Ukkonen,ST-Naive
100,0.000000,0.002010,0.000000,0.001995,0.001000
1000,0.001001,0.017996,0.003000,0.019000,0.011999
10000,0.040003,0.254999,0.060001,0.172000,0.118998
100000,39.630001,3.878001,0.715998,2.727005,2.072996


In [8]:
d={}
dall={}
for size in [1000,10000,100000]:
    d[size] = {}
    dall[size]={}
    d[size]['SA'],d[size]['ST'] = queryOne(size=size)
    dall[size]['SA'],dall[size]['ST'] = queryAll(size=size)

In [13]:
import pandas as pd
pd.DataFrame(d).transpose()

,SA,ST
1000,0.154000,0.808998
10000,0.209001,7.541999
100000,0.474000,68.485126


In [14]:
pd.DataFrame(dall).transpose()

,SA,ST
1000,0.187002,0.817998
10000,0.347999,6.838996
100000,0.900002,65.466998
